In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [7]:
lstm = nn.LSTM(3, 3) #input dim 3, output dim 3
inputs = [torch.randn(1, 3) for _ in range(5)] #make a seq of 5
print('input1', inputs, '\n')

#init hidden state
hidden = (
    torch.randn(1, 1, 3),
    torch.randn(1, 1, 3)
)

inputs = torch.cat(inputs).view(len(inputs), 1, -1)
print('input2', inputs, '\n')
hidden = (torch.randn(1, 1, 3), torch.randn(1, 1, 3))
print('hidden', hidden, '\n')
out, hidden = lstm(inputs, hidden)
print('out', out, '\n')
print('hidden', hidden, '\n')

input1 [tensor([[ 0.5119, -0.6368,  1.0429]]), tensor([[ 0.4903,  1.0318, -0.5989]]), tensor([[ 1.6015, -1.0735, -1.2173]]), tensor([[ 0.6472, -0.0412, -0.1775]]), tensor([[-0.5000,  0.8673, -0.2732]])] 

input2 tensor([[[ 0.5119, -0.6368,  1.0429]],

        [[ 0.4903,  1.0318, -0.5989]],

        [[ 1.6015, -1.0735, -1.2173]],

        [[ 0.6472, -0.0412, -0.1775]],

        [[-0.5000,  0.8673, -0.2732]]]) 

hidden (tensor([[[ 0.0276,  0.5652, -0.0115]]]), tensor([[[ 0.6706, -0.4929,  1.5050]]])) 

out tensor([[[ 0.2490, -0.0525,  0.3253]],

        [[ 0.1655, -0.0304,  0.3348]],

        [[-0.1104, -0.1085,  0.7568]],

        [[-0.0148, -0.0855,  0.4162]],

        [[ 0.0703, -0.1089,  0.2071]]], grad_fn=<StackBackward>) 

hidden (tensor([[[ 0.0703, -0.1089,  0.2071]]], grad_fn=<StackBackward>), tensor([[[ 0.2099, -0.3541,  0.9947]]], grad_fn=<StackBackward>)) 



In [3]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

In [48]:
training_data = [
    ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
    ("Everybody read that book".split(), ["NN", "V", "DET", "NN"]),
    (["book"], ["NN"])
]
word_to_ix = {}
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
print(word_to_ix)
tag_to_ix = {"DET": 0, "NN": 1, "V": 2}

EMBEDDING_DIM = 6
HIDDEN_DIM = 6

{'The': 0, 'dog': 1, 'ate': 2, 'the': 3, 'apple': 4, 'Everybody': 5, 'read': 6, 'that': 7, 'book': 8}


In [49]:
inputs = prepare_sequence(training_data[2][0], word_to_ix)
targets = prepare_sequence(training_data[2][1], tag_to_ix)
print(inputs, targets)

tensor([8]) tensor([1])


In [40]:
class LSTMTagger(nn.Module):
    
    def __init__(
        self,
        embedding_dim,
        hidden_dim,
        vocab_size,
        target_size
    ):
        super(LSTMTagger, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.hidden2tag = nn.Linear(hidden_dim, target_size)
        self.sig = nn.Sigmoid()
    
    def forward(self, sentence):
        print('sentence (per word)', sentence, '\n')
        embeds = self.word_embeddings(sentence)
        print('embed', embeds, '\n')
        print('embed view', embeds.view(len(sentence), 1, -1), '\n')
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        print('lstm out', lstm_out, '\n')
        print('lstm out view', lstm_out.view(len(sentence), -1), '\n')
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        print('tag_space', tag_space, '\n')
#         tag_scores = F.log_softmax(tag_space, dim=1)
        tag_scores = self.sig(tag_space)
        print('tag_scores', tag_scores, '\n')
        return tag_scores

In [59]:
model = LSTMTagger(
    EMBEDDING_DIM,
    HIDDEN_DIM,
    len(word_to_ix),
    len(tag_to_ix)
)
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [57]:
with torch.no_grad():
    inputs = prepare_sequence(training_data[2][0], word_to_ix)
    tag_scores = model(inputs)

sentence (per word) tensor([8]) 

embed tensor([[ 0.6603, -1.0134, -0.6986,  2.0384,  1.8507,  0.3487]]) 

embed view tensor([[[ 0.6603, -1.0134, -0.6986,  2.0384,  1.8507,  0.3487]]]) 

lstm out tensor([[[ 0.3204,  0.0628, -0.0754, -0.0206, -0.1037, -0.0392]]]) 

lstm out view tensor([[ 0.3204,  0.0628, -0.0754, -0.0206, -0.1037, -0.0392]]) 

tag_space tensor([[-0.0439, -0.1832,  0.0726]]) 

tag_scores tensor([[0.4890, 0.4543, 0.5181]]) 



In [60]:
for epoch in range(1):
    for sentence, tags in training_data:
        model.zero_grad()
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)
        
        tag_scores = model(sentence_in)
        print('targets', targets)
        loss = loss_function(tag_scores, targets)
        print('loss', loss)
        
        loss.backward()
        optimizer.step()

sentence (per word) tensor([0, 1, 2, 3, 4]) 

embed tensor([[-0.7096, -0.9373, -0.8042,  0.6697, -0.8039, -0.9156],
        [-0.4856, -0.7153, -0.7518, -0.0544, -1.0632, -2.4394],
        [-0.8824,  0.2072, -2.1338, -0.3505,  0.9777,  1.1637],
        [ 0.0376,  0.6117,  0.3842, -0.7206, -2.8958, -0.2158],
        [ 0.3358, -0.7199,  0.7512, -1.0954, -0.5538,  1.4367]],
       grad_fn=<EmbeddingBackward>) 

embed view tensor([[[-0.7096, -0.9373, -0.8042,  0.6697, -0.8039, -0.9156]],

        [[-0.4856, -0.7153, -0.7518, -0.0544, -1.0632, -2.4394]],

        [[-0.8824,  0.2072, -2.1338, -0.3505,  0.9777,  1.1637]],

        [[ 0.0376,  0.6117,  0.3842, -0.7206, -2.8958, -0.2158]],

        [[ 0.3358, -0.7199,  0.7512, -1.0954, -0.5538,  1.4367]]],
       grad_fn=<ViewBackward>) 

lstm out tensor([[[-0.1965, -0.0402,  0.3115, -0.0392, -0.2654,  0.1014]],

        [[-0.3489, -0.0262,  0.5898, -0.2714, -0.3929,  0.0801]],

        [[-0.0522, -0.4400,  0.1059,  0.0604, -0.2535, -0.1064]],



In [55]:
with torch.no_grad():
    inputs = prepare_sequence(training_data[2][0], word_to_ix)
    tag_scores = model(inputs)
    print(tag_scores)

sentence (per word) tensor([8]) 

embed tensor([[ 1.2678, -1.3178,  2.1215, -2.3513, -0.3042, -1.2749]]) 

embed view tensor([[[ 1.2678, -1.3178,  2.1215, -2.3513, -0.3042, -1.2749]]]) 

lstm out tensor([[[-0.0943,  0.1698,  0.0422, -0.1209, -0.0135, -0.3966]]]) 

lstm out view tensor([[-0.0943,  0.1698,  0.0422, -0.1209, -0.0135, -0.3966]]) 

tag_space tensor([[-0.0651,  0.0510,  0.4032]]) 

tag_scores tensor([[0.4837, 0.5128, 0.5995]]) 

tensor([[0.4837, 0.5128, 0.5995]])
